#### 고려대한국어대사전 오픈프로 포멧으로 변경 (idiom)

1. 오픈프로 필요 기타 칼럼 추가
2. 표제어 자리에 들어갈 부분 추출 : < > 제거 
3. 표제어의 설명부분 숫자별로 분류 (분류해서 새로운 행을 생성해야함)
4. 설명 부분의 < > 부분 용법 내용으로 분류 
5. 품사 폰트 변경
6. "¶" 이걸 기준으로 설명과 예시 분류
7. 용례 유의어 따로 정리 ex> <유> 탈(을) 쓰다.
8. 용례 여러개일 경우 분류
9. 뜻풀이 참조어 링크 태그로 변환 + <유> 부분 태그 변환 (EID 정보 있을 경우)
10. 이형태 정보 
11. 중복제거
12. 네이버와 아이디 매칭

In [2]:
import pandas as pd
import re
from time import time

start_time = time()

pattern_format = {'ꄍ' : '(명사)', 'ꄉ' : '(동사)', 'ꄺ' : '(형용사)', 'ꄖ' : '(부사)'}
symbols = ['①', '②', '③']

# input_file_path 부분을 채워주세요.
input_file_path = "../asset/idiom.csv"
target = pd.read_csv(input_file_path)

In [3]:
# 1. 오픈프로 필요한 기타 칼럼 추가 
target['type'] = '숙어'
target['typediff'] = '괄호 없앤 속담/관용구'
target['shoulder'] = int(0)
target['onum'] = int(1)
target = target[['uid', 'eid', 'ohw', 'ocol', 'idiom', 'shoulder', 
                'type', 'typediff', 'explan', 'onum']]

In [4]:
# 2. 표제어 자리에 들어갈 부분 추출 : < > 제거 
LEN = len(target) 
for i in range(LEN):
    target.loc[i, 'idiom'] = target.loc[i, 'idiom'].replace('◁', '')
    target.loc[i, 'idiom'] = target.loc[i, 'idiom'].replace('▷', '')
    target.loc[i, 'idiom'] = target.loc[i, 'idiom'].strip()

In [5]:
# 3. 표제어의 설명부분 숫자별로 분류 (분류해서 새로운 행을 생성해야함)
extra_list = []
LEN = len(target)
for i in range(LEN):
    if symbols[0] in target.loc[i, 'explan']:
        temps = target.loc[i, 'explan'].split(symbols[1])
        target.loc[i, 'explan'] = temps[0][1:].strip()
        new_row = target.loc[i, :].copy()
        new_row['onum'] = 2
        new_row['explan'] = temps[1].strip()
        extra_list.append(new_row)

LEN2 = len(extra_list)
for i in range(LEN2):
    new_row = pd.DataFrame({'uid':[extra_list[i]['uid']], 'eid':[extra_list[i]['eid']], 'ohw':[extra_list[i]['ohw']], 'ocol':[extra_list[i]['ocol']], 
                            'idiom':[extra_list[i]['idiom']], 'shoulder':[extra_list[i]['shoulder']], 'type':[extra_list[i]['type']], 
                            'typediff':[extra_list[i]['typediff']], 'explan':[extra_list[i]['explan']], 'onum':[extra_list[i]['onum']]})
    target = pd.concat([target, new_row], ignore_index=True)

target = target.sort_values(by='uid', kind='stable')
target = target.reset_index(drop=True)

extra_list = []
LEN = len(target)
for i in range(LEN):
    if symbols[2] in target.loc[i, 'explan']:
        temps = target.loc[i, 'explan'].split(symbols[2])
        target.loc[i, 'explan'] = temps[0].strip()
        new_row = target.loc[i, :].copy()
        new_row['explan'] = temps[1].strip()
        new_row['onum'] = 3
        extra_list.append(new_row)

LEN2 = len(extra_list)
for i in range(LEN2):
    new_row = pd.DataFrame({'uid':[extra_list[i]['uid']], 'eid':[extra_list[i]['eid']], 'ohw':[extra_list[i]['ohw']], 'ocol':[extra_list[i]['ocol']], 
                            'idiom':[extra_list[i]['idiom']], 'shoulder':[extra_list[i]['shoulder']], 'type':[extra_list[i]['type']], 
                            'typediff':[extra_list[i]['typediff']], 'explan':[extra_list[i]['explan']], 'onum':[extra_list[i]['onum']]})
    target = pd.concat([target, new_row], ignore_index=True)

target = target.sort_values(by='uid', kind='stable')
target = target.reset_index(drop=True)

In [6]:
# 4. 설명 부분의 < > 부분 용법 내용으로 분류 
target['pattern'] = ''

LEN = len(target)
for i in range(LEN):
    p = r'\s*◀(.*?)▶' 
    m = re.search(p, target.loc[i, 'explan'])
    if m:
        result = m.group().strip() 
        target.loc[i, 'pattern'] = result[1:-1] 
        target.loc[i, 'explan'] = re.sub(p, '', target.loc[i, 'explan']).strip() 

In [7]:
# 5. 품사 폰트 변경
LEN = len(target)
for i in range(LEN):
    if target.loc[i, 'pattern'] == '':
        continue
    for pattern in pattern_format:
        target.loc[i, 'pattern'] = re.sub(pattern, pattern_format[pattern], target.loc[i, 'pattern'])

target = target[['uid', 'eid', 'ohw', 'ocol', 'idiom', 'shoulder', 'type', 'typediff', 'onum', 'pattern', 'explan']]

In [8]:
# 6. "¶" 이걸 기준으로 설명과 예시 분류
LEN = len(target)
target['explan'] = target['explan'].str.strip()
for i in range(LEN):
    temp = target.loc[i, 'explan'].split('¶')
    target.loc[i, 'explan'] = temp[0].strip()
    if len(temp) != 1:
        target.loc[i, 'example'] = temp[1].strip()

In [9]:
# 7. 용례 유의어 따로 정리 ex> <유>, <반> 탈(을) 쓰다. (뜻번호당 하나만!!! 용례당 하나가 아님)
target['example'].fillna('', inplace=True)
target['sim_type'] = ''
target['similar_expression'] = ''

extra_list = []

LEN = len(target)
for i in range(LEN):
    if '<유>' in target.loc[i, 'example'] and '<반>' not in target.loc[i, 'example']: 
        temps = target.loc[i, 'example'].split('<유>')
        target.loc[i, 'example'] = temps[0]
        target.loc[i, 'similar_expression'] = temps[1].strip()
        target.loc[i, 'sim_type'] = '유의어'
    elif '<유>' not in target.loc[i, 'example'] and '<반>' in target.loc[i, 'example']: 
        temps = target.loc[i, 'example'].split('<반>')
        target.loc[i, 'example'] = temps[0]
        target.loc[i, 'similar_expression'] = temps[1].strip()
        target.loc[i, 'sim_type'] = '반의어'
    elif '<유>' in target.loc[i, 'example'] and '<반>' in target.loc[i, 'example']: 
        temps = target.loc[i, 'example'].split('<유>')
        temps = temps[1].split('<반>')
        target.loc[i, 'sim_type'] = '유의어'
        target.loc[i, 'similar_expression'] = temps[0]
        new_row = target.loc[i, :].copy()
        new_row['sim_type'] = '반의어'
        new_row['similar_expression'] = temps[1].strip()
        extra_list.append(new_row)
        

LEN2 = len(extra_list)
for i in range(LEN2):
    new_row = pd.DataFrame({'uid':[extra_list[i]['uid']], 'eid':[extra_list[i]['eid']], 'ohw':[extra_list[i]['ohw']], 'ocol':[extra_list[i]['ocol']], 
                            'idiom':[extra_list[i]['idiom']], 'shoulder':[extra_list[i]['shoulder']], 'type':[extra_list[i]['type']], 
                            'typediff':[extra_list[i]['typediff']], 'onum':[extra_list[i]['onum']], 'pattern':[extra_list[i]['pattern']], 
                            'explan':[extra_list[i]['explan']], 'example':extra_list[i]['example'], 'sim_type':[extra_list[i]['sim_type']],
                            'similar_expression':[extra_list[i]['similar_expression']]})
    target = pd.concat([target, new_row], ignore_index=True)

target = target.sort_values(by='uid', kind='stable')
target = target.reset_index(drop=True)

/var/folders/f4/dlc6vs952d9_s55nyfp4qb8r0000gn/T/ipykernel_65176/2173395775.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  target['example'].fillna('', inplace=True)


In [10]:
# 8. 용례 여러개일 경우 분류
LEN = len(target)
extra_list = []
for i in range(LEN):
    if '/' in target.loc[i, 'example']:
        temps = target.loc[i, 'example'].split(' / ')
        target.loc[i, 'example'] = temps[0].strip()
        for example in temps[1:]:
            new_row = target.loc[i, :].copy()
            new_row['example'] = example.strip()
            extra_list.append(new_row)

LEN2 = len(extra_list)
for i in range(LEN2):
    new_row = pd.DataFrame({'uid':[extra_list[i]['uid']], 'eid':[extra_list[i]['eid']], 'ohw':[extra_list[i]['ohw']], 'ocol':[extra_list[i]['ocol']], 
                            'idiom':[extra_list[i]['idiom']], 'shoulder':[extra_list[i]['shoulder']], 'type':[extra_list[i]['type']], 
                            'typediff':[extra_list[i]['typediff']], 'onum':[extra_list[i]['onum']], 'pattern':[extra_list[i]['pattern']], 
                            'explan':[extra_list[i]['explan']], 'example':extra_list[i]['example']})
    target = pd.concat([target, new_row], ignore_index=True)

target['sim_type'].fillna('', inplace=True)
target['similar_expression'].fillna('', inplace=True)
target = target.sort_values(by='uid', kind='stable')
target = target.reset_index(drop=True)

/var/folders/f4/dlc6vs952d9_s55nyfp4qb8r0000gn/T/ipykernel_65176/4134737423.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  target['sim_type'].fillna('', inplace=True)
/var/folders/f4/dlc6vs952d9_s55nyfp4qb8r0000gn/T/ipykernel_65176/4134737423.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always 

In [11]:
# 9. 뜻풀이 참조어 링크 태그로 변환 + <유>

In [12]:
# 10. 이형태 정보
LEN = len(target)
pattern1 = '\((.*?)\)'
pattern2 = '〔(.*?)〕'
for i in range(LEN):
    if target.loc[i, 'idiom'] != "":
        target.loc[i, 'idiomdiff'] = re.sub(pattern1, "", target.loc[i, 'idiom'])
        target.loc[i, 'idiomdiff'] = re.sub(pattern2, "", target.loc[i, 'idiomdiff'])

target = target[['uid', 'eid', 'ohw', 'ocol', 'idiom', 'idiomdiff', 'shoulder', 'type',
        'typediff', 'onum', 'pattern', 'explan', 'example', 'sim_type',
        'similar_expression']]


In [13]:
# 11. 중복제거
condition1 = target.duplicated(subset=['uid', 'onum', 'example'], keep='first')
condition2 = target.duplicated(subset=['uid', 'onum'], keep='first')
condition3 = target.duplicated(subset=['uid'], keep='first')
condition_final = target.duplicated()
target.loc[condition1, 'uid':'example'] = ''
target.loc[condition2, 'uid':'explan'] = ''
target.loc[condition3, 'uid':'typediff'] = ''
target = target.drop(target[condition_final].index)
target = target.reset_index(drop=True)

/var/folders/f4/dlc6vs952d9_s55nyfp4qb8r0000gn/T/ipykernel_65176/485847252.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  target.loc[condition1, 'uid':'example'] = ''
/var/folders/f4/dlc6vs952d9_s55nyfp4qb8r0000gn/T/ipykernel_65176/485847252.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  target.loc[condition1, 'uid':'example'] = ''
/var/folders/f4/dlc6vs952d9_s55nyfp4qb8r0000gn/T/ipykernel_65176/485847252.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  target.

In [14]:
# 12. 네이버와 아이디 매칭
target['naver_id'] = ""
target = target[['uid', 'naver_id', 'eid', 'ohw', 'ocol', 'idiom', 'idiomdiff', 'shoulder', 'type', 'typediff',
                'onum', 'pattern', 'explan', 'example', 'sim_type', 'similar_expression']]

ID = pd.read_csv('../asset/idiom_id.csv')
ID.rename(columns={'naver_ID':'naver_id'}, inplace=True)
ID['is_matched'] = 'dismatched'
LEN = len(target)
LEN_ID = len(ID)
for i in range(LEN):
        target_idiom = target.loc[i, 'idiom']
        start = 0
        end = LEN_ID - 1
        while True:
                if start > end:
                        break
                mid = (start + end) // 2
                try_idiom = ID.loc[mid, 'idiom']
                if target_idiom == try_idiom:
                        target.loc[i, 'naver_id'] = ID.loc[mid, 'naver_id']
                        ID.loc[mid, 'is_matched'] = 'matched'
                        break
                elif target_idiom < try_idiom:
                        end = mid - 1
                else:
                        start = mid + 1

In [15]:
condition = (target['naver_id'] == "") & (target['uid'] != "")
# 신규어 개수 (매칭 안된 idiom)
print(f"신규(매칭 안됨) : {len(target[condition])}")

신규(매칭 안됨) : 2


In [16]:
# target.to_csv('../result/processed_idiom.csv', index=False)
# end_time = time()
# print(f"!!!FINISHED!!! 걸린시간 : {round(end_time - start_time, 2)}s")

In [17]:
target[condition]

,uid,naver_id,eid,ohw,ocol,idiom,idiomdiff,shoulder,type,typediff,onum,pattern,explan,example,sim_type,similar_expression
5239,4978,,52189,길거리,길거리로나앉다,길거리로〔길거리에〕 나앉다,길거리로 나앉다,0,숙어,괄호 없앤 속담/관용구,1,(명사)이,(사람이) 머물 곳도 마련하지 못할 정도로 가난해지다.,국환이는 도박에 빠져서 재산을 탕진하고 길거리에 나앉았다.,유의어,거리로 나앉다.
5609,5280,,267972,입,입을대다,입(을) 대다,입 대다,0,숙어,괄호 없앤 속담/관용구,1,(명사)이,(사람이) 어떤 대상에 대하여 평가하거나 간섭하다.,"조교에게 자료 정리를 시켰는데, 어느 한 구석 입 댈 필요가 없을 정도로 깔끔하게 ...",,


In [18]:
condition2 = ID['is_matched'] == 'dismatched'
ID[condition2]

,naver_id,type,idiom,shoulder,is_matched
130,9bd45443602641989dc052ad9c580ced,숙어,거리로 나앉다 / 거리에 나앉다,0,dismatched
139,b5ea11193c5a4d9795a2d13103c35447,숙어,걱정도 팔자다,0,dismatched
147,68dc8795b1774ed8ad05186fb03bfad9,숙어,걸신(이) 들린 듯이,0,dismatched
334,893ff987677145c9964262c2b65319c5,숙어,귀에 못이 박히다,0,dismatched
429,928316f6b23c497c8247a39abef8ea85,숙어,길거리로 나앉다 / 길거리에 나앉다,0,dismatched
515,28a6868d81134c6aa26fe118ca0cda77,숙어,나 죽었소 하다,0,dismatched
672,f820835150324180856348b9c58430a8,숙어,눈물(이) 없이,0,dismatched
691,d97f8fb101f043ee831a1866018b179e,숙어,눈앞이 캄캄하다,0,dismatched
766,15353c90aec34c41b04a44e1f16b5f65,숙어,달게 굴다,0,dismatched
938,f4edb853c33f4c5fb79bbfbb17964397,숙어,뒷배(를) 보다,0,dismatched


In [19]:
cnt = 0
LEN = len(target)
for i in range(LEN):
    if 'EID' in target.loc[i, 'example']:
        print(target.loc[i, 'example'])